In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time
import gc
from csv import writer

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
#from skimage import io

import torch.optim as optim
import torch.autograd as autograd

In [4]:
import sys
#sys.path.append('/content/GCIT-1')
from GCIT import *
from utils import *
from modelos_gancito import *
from entrenamiento_gan_cit_0 import *

1.5.1


In [5]:
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

In [7]:
Zdim = 500 # 25, 50, 100, 250, 500
dim_x = 1
dim_y = 1 
# fijar número de experimentos a realizar
n_experiments = 42 #faltan 900
# variable para guardar los pvalores de la función GCIT-1
p_vals= []
p_values = []
# determinar de manera aleatoria cual es el tipo de conjunto a evaluar
true = np.random.randint(0,2,n_experiments)
print('Probando test para '+str(Zdim)+' dimensiones de Z')
pbar = tqdm(total=n_experiments)
for n in range(n_experiments):
    ti = time.time()
    # CREAR DATASET
    if true[n]==0:
        data_type = 'CI' # conjunto condicionalmente independiente
    if true[n]==1:
        data_type = 'NI' # conjunto dependiente 
    x, y, z = generate_samples_random(size=500, sType=data_type,
                                      dx=dim_x, dy=dim_y, dz=Zdim,
                                      fixed_function='linear', dist_z='gaussian')
    # entrenar gancito y obtener modelo GAN entrenado
    zMI_loss, xMI_loss, G_loss, D_loss, G_net = training(x,y,z,'GCIT_InfoGAN', 2e-4) 
    # REALIZAR TEST
    n_samples = 1000
    p_vals = [] # para guardar los valores de los 5 estadísticos
    statistics = [correlation, mmd_squared, kolmogorov, wilcox, rdc]
    dim_v = 62
    for stat in statistics:
        rho = []
        for sample in range(n_samples):
            v = torch.FloatTensor(500, dim_v)
            x_hat = G_net.forward(torch.tensor(z).float(), v.uniform_(0,1))
            x_hat = x_hat.cpu().detach().numpy()
            rho.append(stat(x_hat, y))
        #print(rho)
        rho = np.array(rho)
        p_value = min(sum(rho < stat(x.reshape(len(x)),y)) / n_samples, sum(rho > stat(x.reshape(len(x)), y)) / n_samples)
        p_vals.append(p_value)
    #p_vals = GCIT_1(x, y, z, statistic='all')
    del x, y, z
    pbar.update(1)
    # GUARDAR RESULTADOS
    append_list_as_row('dim%s_linear_gaussian_GANCIT0.csv'%(Zdim,), [true[n].tolist()] + p_vals)
    gc.collect()
p_values = np.array(p_values)
pbar.close()
# valita cool = cool

Probando test para 500 dimensiones de Z
